In [12]:
import math
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

**This is the start of Tutorial 2**

In [13]:
## Define Variables

B = 150 # channel width (m)
H = 8 # flow depth (m)
S = 0.0002 # channel slope (m/m)
Cf = 60 # Chezy friction coefficient
D50 = 0.001 # median grain size (m)
shareOfIjssel = 0.17
Ql = 3200*shareOfIjssel # minimum discharge (m^3/s)
Qh = 20000*shareOfIjssel # maximum discharge (m^3/s)
R = 1.65 # submerged specific gravity of sediment
g = 9.81 # gravitational acceleration (m/s^2)
rho = 1000 # water density (kg/m^3)

u_max = Cf*np.sqrt(H*S) # main channel flow velocity (m/s)
A = B*H # main channel cross-sectional area (m^2)
P = B + 2*H # main channel wetted perimeter (m)
Rh = A/P # main channel hydraulic radius (m)

print(maximum_discharge := u_max*A) # maximum discharge (m^3/s)
print(Qh-maximum_discharge)

h_safety = 1 
h_cmax = H-h_safety # flow depth during design discharge (m)
u_cmax = Cf*np.sqrt(h_cmax*S) # flow velocity during design discharge (m/s)
A_cmax = B*h_cmax # cross-sectional area during design discharge (m^2)
Q_cmax = u_cmax*A_cmax # design discharge (m^3/s)
print("Maximum main channel capacity with safety",Q_cmax) # design discharge (m^3/s)

# design for main channel
# set by height



x = (1043/Cf/np.sqrt(S))**(2/5)
print(x)



2880.0
520.0000000000005
Maximum main channel capacity with safety 2357.2441536675833
17.21262963367693
